In [1]:
!pip install requests
!pip install xlrd

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import datetime as dt
# common imports
import zipfile
import time
# import datetime, timedelta
import datetime
from datetime import datetime, timedelta
from datetime import date
from dateutil import relativedelta
from io import StringIO
import pandas as pd
import pickle
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from io import StringIO
import requests
import json
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline 
import os
import math
from subprocess import check_output
from IPython.display import display
import logging
import yaml
from collections import Counter
import re
import os


In [3]:
# common variables

load_from_scratch = True

# control whether to save dataframe with transformed data
save_transformed_dataframe = True

# control whether rows containing erroneous values are removed from the saved dataset
remove_bad_values = True

# name of file containing pickled dataframe version of input (unprocessed) dataset
pickled_input_dataframe = "2014_2020.pkl"

# name of file to which prepared data set is saved as a pickled dataframe
pickled_output_dataframe = "cleaned_Data.pkl"


In [4]:
streetcar_vehicles = list(range(4000,4006))+ list(range(4010,4200)) +  list(range(4200,4252)) + [4900]
streetcar_vehicles = streetcar_vehicles + [4400] + list(range(4402,4508))
#print("valid streetcars",streetcar_vehicles)

In [5]:
bus_vehicles = list(range(1000,1150))+ list(range(2000,2111)) + list(range(2150,2156)) + list(range(2240,2486))
bus_vehicles = bus_vehicles + list(range(2600,2620)) + list(range(2700,2766)) + list(range(2767,2859))
bus_vehicles = bus_vehicles + list(range(7000,7135)) + list(range(7400,7450)) + list(range(7500,7620)) + list(range(7620,7882))
bus_vehicles = bus_vehicles + list(range(8000,8100)) + list(range(9000,9027))
valid_vehicles = streetcar_vehicles + bus_vehicles
#Using ony valid vehicles

In [6]:
# load the valid list of TTC Streetcar routes
valid_routes = ['501','502','503','504','505','506','509','510','511','512','301','304','306','310']

In [7]:
# original valid directions
# valid_directions = ['E/B','W/B','N/B','S/B','B/W']
# revised valid directions to include lowercasing and removal of '/' and simplify to single letter
valid_directions = ['e','w','n','s','b']

In [8]:
valid_days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

In [9]:
# get the directory for that this notebook is in and return the directory containing data files

def get_path():
    rawpath = os.getcwd()
    # data is in a directory called "data" that is a sibling to the directory containing the notebook
    path = os.path.abspath(os.path.join(rawpath, '..', 'data'))
    return(path)

# given a path return the list of xls files in the directory
def get_xls_list(path):
    files = os.listdir(path)
    files_xls = [f for f in files if f[-4:] == 'xlsx']
    print(files)
    print(files_xls)
    return(files_xls)

def load_xls(path, files_xls, firstfile, firstsheet, df):
    for f in files_xls:
        print("file name",f)
        xlsf = pd.ExcelFile(os.path.join(path,f))
        # iterate through sheets
        for sheet_name in xlsf.sheet_names:
            print("sheet_name",sheet_name)
            if (f != firstfile) or (sheet_name != firstsheet):
                print("sheet_name in loop",sheet_name)
                data = pd.read_excel(os.path.join(path,f),sheet_name=sheet_name)    
                df = df.append(data)
    return (df)

# given a path and a filename, load all the XLS files in the path into a dataframe and save
# to the dataframe to the filename
def reloader(path,picklename):
    # get list of all xls files in the path
    files_xls = get_xls_list(path)
    print("list of xls",files_xls)
    # seed initial tab on initial xls file
    dfnew = pd.read_excel(os.path.join(path,files_xls[0]))
    # get the list of sheets in the first file
    xlsf = pd.ExcelFile(os.path.join(path,files_xls[0]))
    # load the remaining tabs from all the other xls
    # pass the first file (files_xls[0]) and the first tab in that file (xlsf[0]) explicitly
    dflatest = load_xls(path,files_xls,files_xls[0],xlsf.sheet_names[0], dfnew)
    # save dataframe to pickle
    dflatest.to_pickle(os.path.join(path,picklename))
    # return dataframe loaded with all tabs of all xls files
    return(dflatest)
    
    # define categories for input columns
def define_feature_categories(df):
    allcols = list(df)
    print("all cols",allcols)
    textcols = ['Incident','Location'] # 
    continuouscols = ['Min Delay','Min Gap'] 
                      # columns to deal with as continuous values - no embeddings
    timecols = ['Report Date','Time']
    collist = ['Day','Vehicle','Route','Direction']
    for col in continuouscols:
        df[col] = df[col].astype(float)
    print('texcols: ',textcols)
    print('continuouscols: ',continuouscols)
    print('timecols: ',timecols)
    print('collist: ',collist)
    return(allcols,textcols,continuouscols,timecols,collist)
# fill missing values according to the column category
def fill_missing(dataset,allcols,textcols,continuouscols,timecols,collist):
    logging.debug("before mv")
    for col in collist:
        dataset[col].fillna(value="missing", inplace=True)
    for col in continuouscols:
        dataset[col].fillna(value=0.0,inplace=True)
    for col in textcols:
        dataset[col].fillna(value="missing", inplace=True)
    return (dataset)
# read in data, either from original XLS files in data directory or from pickled dataframe containing
def ingest_data(path):
    if load_from_scratch:
        unpickled_df = reloader(path,pickled_input_dataframe)
        logging.debug("reloader done")
    else:
        unpickled_df = pd.read_pickle(os.path.join(path,pickled_input_dataframe))
    return(unpickled_df)

# General cleanup
- correct types for Route and Vehicle
- fill missing values
- create report-date-time index

In [10]:
# the dataset incorporated some anomalies in the 2019 data, including:
# extraneous Incident ID in April 2019 tab
# Gap and Delay columns in April and June 2019 tabs for what had otherwise been called Min Gap and Min Delay
# this function cleans up these anomalies
def fix_anomalous_columns(df):
    # for rows where there is NaN in the Min Delay or Min Gap columns, copy over value from Delay or Gap
    # df.Temp_Rating.fillna(df.Farheit, inplace=True)
    df['Min Delay'].fillna(df['Delay'], inplace=True)
    df['Min Gap'].fillna(df['Gap'], inplace=True)
    # now that the useful values have been copied from Delay and Gap, remove them
    del df['Delay']
    del df['Gap']
    # remove Incident ID column - it's extraneous
    del df['Incident ID']
    return(df)

def replace_time(date_time_value,time_value):
    ''' given a datetime replace the time portion '''
     
    date_time_value = date_time_value.replace(hour=time_value.hour,minute=time_value.minute,second=time_value.minute)
    return(date_time_value)

def general_cleanup(df):
    # ensure Route and Vehicle are strings, not numeric
    df['Route'] = df['Route'].astype(str)
    df['Vehicle'] = df['Vehicle'].astype(str)
    # remove extraneous characters left from Vehicle values being floats
    df['Vehicle'] = df['Vehicle'].str[:-2]
    # tactical definition of categories
    allcols,textcols,continuouscols,timecols,collist = define_feature_categories(df)
    # fill in missing values
    df.isnull().sum(axis = 0)
    df = fix_anomalous_columns(df)
    df = fill_missing(df,allcols,textcols,continuouscols,timecols,collist)
    # create new column combining date + time (needed for resampling) and make it the index
    df['Report Date Time'] = df.apply(lambda x: replace_time(x['Report Date'], x['Time']), axis=1)
    df.index = df['Report Date Time']
    # return the updated dataframe along with the column category lists
    return(df,allcols,textcols,continuouscols,timecols,collist)

# Cleaning Route, Vehicle, Direction and Location

In [11]:
#-------------------------------------------------------
#Functions for cleaning the route columns
def check_route (x):
    if x in valid_routes:
        return(x)
    else:
        return("bad route")
def route_cleanup(df):
    print("Route count pre cleanup",df['Route'].nunique())
    # df['Route'].value_counts()
    # replace bad route with common token
    df['Route'] = df['Route'].apply(lambda x:check_route(x))
    print("route count post cleanup",df['Route'].nunique())
    return(df)    


#-------------------------------------------------------
#Functions for checking and cleaning the vehicle columns
def check_vehicle (x):
    if str.isdigit(x):
        if int(x) in valid_vehicles:
            return x
        else:
            return("bad vehicle")
    else:
        return("bad vehicle")
    
def vehicle_cleanup(df):
    print("Vehicle count pre cleanup",df['Vehicle'].nunique())
    df['Vehicle'] = df['Vehicle'].apply(lambda x:check_vehicle(x))
    print("Vehicle count post cleanup",df['Vehicle'].nunique())
    return(df)


#-------------------------------------------------------
#Functions for cleaning the direction columns
def check_direction (x):
    if x in valid_directions:
        return(x)
    else:
        return("bad direction")
    
def direction_cleanup(df):
    print("Direction count pre cleanup",df['Direction'].nunique())
    df['Direction'] = df['Direction'].str.lower()
    df['Direction'] = df['Direction'].str.replace('/','')
    df['Direction'] = df['Direction'].replace({'eastbound':'e','westbound':'w','southbound':'s','northbound':'n'})
    df['Direction'] = df['Direction'].replace('b','',regex=True)
    df['Direction'] = df['Direction'].apply(lambda x:check_direction(x))
    print("Direction count post cleanup",df['Direction'].nunique())
    return(df)


#-------------------------------------------------------
#Functions for cleaning the Location columns
def clean_conjunction(intersection):
    intersection = re.sub(" *& *"," and ",intersection)
    intersection = re.sub(" */ *"," and ",intersection)
    return(intersection)

def order_location(intersection):
    # for any string with the format "* and *" if the value before the and is alphabetically
    # higher than the value after the and, swap the values
    conj = " and "
    alpha_ordered_intersection = intersection
    if conj in intersection:
        end_first_street = intersection.find(conj)
        if (end_first_street > 0) and (len(intersection) > (end_first_street + len(conj))):
            start_second_street = intersection.find(conj) + len(conj)
            first_street = intersection[0:end_first_street]
            second_street = intersection[start_second_street:]
            alpha_ordered_intersection = min(first_street,second_street)+conj+max(first_street,second_street)
    return(alpha_ordered_intersection)

def location_cleanup(df):
    print("Location count pre cleanup",df['Location'].nunique())
    # make all location values lower case
    df['Location'] = df['Location'].str.lower()
    # make substitutions to eliminate obvious duplicate tokens
    df['Location'] = df['Location'].replace({'broadviewstation':'broadview station',' at ':' and ',' stn':' station',' ave.':'','/':' and ','roncy':'roncesvalles','carhouse':'yard','yard.':'yard','st. clair':'st clair','ronc. ':'roncesvalles ','long branch':'longbranch','garage':'yard','barns':'yard',' & ':' and '}, regex=True)
    # put intersection values into consistent order
    df['Location'] = df['Location'].apply(lambda x:order_location(x))
    print("Location count post cleanup",df['Location'].nunique())
    return(df)

#-------------------------------------------------------
# remove rows with bad values
def remove_bad(df):
    df = df[df.Vehicle != 'bad vehicle']
    df = df[df.Direction != 'bad direction']
    df = df[df.Route != 'bad route']
    return(df)

In [12]:
# master cell to call the other functions
# get the path for data files
path = get_path()
# load route direction and delay data datframes
df = ingest_data(path)
df,allcols,textcols,continuouscols,timecols,collist = general_cleanup(df)

['2014_2020.pkl', 'cleaned_Data.pkl', 'excessdata', 'routedirection.csv', 'routedirection_badvalues.csv', 'tensorboard_log', 'ttc-streetcar-delay-data-2014.xlsx', 'ttc-streetcar-delay-data-2015.xlsx', 'ttc-streetcar-delay-data-2016.xlsx', 'ttc-streetcar-delay-data-2017.xlsx', 'ttc-streetcar-delay-data-2018.xlsx', 'ttc-streetcar-delay-data-2019.xlsx', 'ttc-streetcar-delay-data-2020.xlsx']
['ttc-streetcar-delay-data-2014.xlsx', 'ttc-streetcar-delay-data-2015.xlsx', 'ttc-streetcar-delay-data-2016.xlsx', 'ttc-streetcar-delay-data-2017.xlsx', 'ttc-streetcar-delay-data-2018.xlsx', 'ttc-streetcar-delay-data-2019.xlsx', 'ttc-streetcar-delay-data-2020.xlsx']
list of xls ['ttc-streetcar-delay-data-2014.xlsx', 'ttc-streetcar-delay-data-2015.xlsx', 'ttc-streetcar-delay-data-2016.xlsx', 'ttc-streetcar-delay-data-2017.xlsx', 'ttc-streetcar-delay-data-2018.xlsx', 'ttc-streetcar-delay-data-2019.xlsx', 'ttc-streetcar-delay-data-2020.xlsx']
file name ttc-streetcar-delay-data-2014.xlsx
sheet_name Jan 201

In [13]:

# get record count by year
from collections import Counter
df_year = pd.DatetimeIndex(df['Report Date Time']).year
print("record count by year pre processing: ", str(Counter(df_year)))

# check that the values for April 2019 are correct as there were anomolaies in this section
df[df['Report Date Time'].astype(str).str[:7]=='2019-04']

# cleanup Route
logging.debug("df.shape output pre route cleanup",df.shape)
df = route_cleanup(df) 
df = vehicle_cleanup(df)
df = direction_cleanup(df)
df = location_cleanup(df)
logging.debug("df.shape output post location",df.shape)
print("Bad route count pre:",df[df.Route == 'bad route'].shape[0])
print("Bad direction count pre:",df[df.Direction == 'bad direction'].shape[0])
print("Bad vehicle count pre:",df[df.Vehicle == 'bad vehicle'].shape[0])
if remove_bad_values:
    df = remove_bad(df)
print("Bad route count:",df[df.Route == 'bad route'].shape[0])
print("Bad direction count:",df[df.Direction == 'bad direction'].shape[0])
print("Bad vehicle count:",df[df.Vehicle == 'bad vehicle'].shape[0])


# pickle the cleansed dataframe in the file named cleaned_Data.pkl
print("df.shape output post removal of bad records ",df.shape)
if save_transformed_dataframe:
    print("path is ",path)
    file_name = os.path.join(path,pickled_output_dataframe)
    print("file_name is ",file_name)
    df.to_pickle(file_name)
df.head()

record count by year pre processing:  Counter({2018: 15612, 2016: 14021, 2017: 13762, 2015: 12221, 2019: 11882, 2014: 11027, 2020: 4177})
Route count pre cleanup 124
route count post cleanup 15
Vehicle count pre cleanup 2727
Vehicle count post cleanup 1020
Direction count pre cleanup 103
Direction count post cleanup 5
Location count pre cleanup 17679
Location count post cleanup 11355
Bad route count pre: 2618
Bad direction count pre: 414
Bad vehicle count pre: 16975
Bad route count: 0
Bad direction count: 0
Bad vehicle count: 0
df.shape output post removal of bad records  (63390, 11)
path is  C:\Users\mshaf\MachineLearningTTC\data
file_name is  C:\Users\mshaf\MachineLearningTTC\data\cleaned_Data.pkl


,Report Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle,Report Date Time
Report Date Time,,,,,,,,,,,
2014-01-02 06:31:31,2014-01-02,505,06:31:00,Thursday,dundas and roncesvalles,Late Leaving Garage,4.0,8.0,e,4018,2014-01-02 06:31:31
2014-01-02 12:43:43,2014-01-02,504,12:43:00,Thursday,king and shaw,Utilized Off Route,20.0,22.0,e,4128,2014-01-02 12:43:43
2014-01-02 14:01:01,2014-01-02,501,14:01:00,Thursday,bingham and kingston road,Held By,13.0,19.0,w,4016,2014-01-02 14:01:01
2014-01-02 14:22:22,2014-01-02,504,14:22:00,Thursday,king st. and roncesvalles,Investigation,7.0,11.0,w,4175,2014-01-02 14:22:22
2014-01-02 16:42:42,2014-01-02,504,16:42:00,Thursday,bathurst and king,Utilized Off Route,3.0,6.0,e,4080,2014-01-02 16:42:42


In [14]:
# get record count by year

df_year = pd.DatetimeIndex(df['Report Date Time']).year
Counter(df_year)

Counter({2014: 9371,
         2015: 10898,
         2016: 11908,
         2017: 9891,
         2018: 12011,
         2019: 7474,
         2020: 1837})